# Historical Aution State Classifier

### The goal:

we take 3 inputs:
From prior session (D-1)
	•	VAH₋₁
	•	VAL₋₁
	•	POC₋₁
From current session (D)
	•	Session open price
	•	Time-ordered price bars (fixed interval)



and give up 2 outputs:
   State
        balance
        attempted_breakout up
        attempted_breakout down
        discovery up
        discovery down
   Segment


we're deliberately ignoring:
	•	Developing value
	•	Volume delta
	•	VWAP
	•	Multi-day context
	•	News
	•	Volatility

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [4]:
def get_data(ticker, period, interval):
    
    #############################
    # input: ticker and interval
    # output: return
    
    ticker = yf.Ticker(str(ticker))
    data = ticker.history(period = str(period), interval = str(interval))
    return data

data = get_data(ticker = '^GSPC', period = 'max', interval = '5m') # ^GSPC is the ticker for the S&P 500

In [ ]:
# one day, if we can calculate the value area for one, we can make a function to do it for them all. 
df = data.head(78)

In [13]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2025-11-03 09:30:00-05:00,6882.319824,6882.319824,6867.720215,6868.649902,0,0.0,0.0
2025-11-03 09:35:00-05:00,6868.810059,6868.810059,6857.819824,6857.819824,68015569,0.0,0.0
2025-11-03 09:40:00-05:00,6858.000000,6858.000000,6851.229980,6852.890137,61679016,0.0,0.0
2025-11-03 09:45:00-05:00,6852.910156,6855.569824,6850.379883,6852.259766,55581160,0.0,0.0
2025-11-03 09:50:00-05:00,6852.229980,6852.229980,6844.660156,6845.520020,53575193,0.0,0.0
...,...,...,...,...,...,...,...
2025-11-03 15:35:00-05:00,6847.799805,6851.500000,6846.160156,6851.500000,49177000,0.0,0.0
2025-11-03 15:40:00-05:00,6851.509766,6853.200195,6850.220215,6852.720215,53752000,0.0,0.0
2025-11-03 15:45:00-05:00,6852.689941,6853.359863,6846.399902,6847.839844,62323000,0.0,0.0
